Написать приложение, которое собирает основные новости с сайтов mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:
название источника;
наименование новости;
ссылку на новость;
дата публикации.

я буду парсить Хабр 

In [140]:
import requests
import json
import pandas as pd
import numpy as np
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint
from lxml import html
import time
from selenium.webdriver.common.by import By
from pymongo import MongoClient

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Андрей\.wdm\drivers\chromedriver\win32\94.0.4606.61]


In [3]:
driver.get('https://habr.com/ru/search/')
source_data = driver.page_source

In [4]:
target = 'AI'

<input autofocus="autofocus" name="q" placeholder="Поиск" value="" class="tm-input-text-decorated__input">

In [5]:
sbox = driver.find_element_by_class_name("tm-input-text-decorated__input")
sbox.send_keys(target)
sbox.submit()

In [16]:
time.sleep(3)

<button class="tm-navigation-dropdown__button">
      по релевантности
      <svg height="16" width="16" class="tm-svg-img icon_dropdown-arrow"><title>Открыть список</title> <use xlink:href="/img/megazord-v24.cee85629.svg#arrow-down"></use></svg></button>

In [6]:
sbox = driver.find_element_by_class_name("tm-navigation-dropdown__button")
sbox.click()

<li class="tm-navigation-dropdown__option">
      по времени
      <!----></li>
      
      //*[@id="app"]/div[1]/div[2]/main/div/div/div/div[1]/div/div[2]/div[2]/ul/li[2]

вот только сейчас дошло что очень удобно здесь использовать xpath, но менять уже ничего не буду, пусть так лежит))

In [8]:
sbox = driver.find_element_by_xpath('//*[@id="app"]/div[1]/div[2]/main/div/div/div/div[1]/div/div[2]/div[2]/ul/li[2]')
sbox.click()

In [136]:
source_data = driver.find_elements(By.XPATH, "//div[@class='tm-article-snippet']")
habr_list = []

In [128]:
sour = driver.find_elements_by_xpath("//article[@class='tm-articles-list__item']//child::span[text()]")

не разобрался как вытаскивать все значения с текстом

In [130]:
for el in sour:
    habr_dict['Name'] = el
    #print(el.text)

daniilshat
daniilshat
Исследователи МТИ научили нейросеть предсказывать аварийные участки дорог
+7
321
321
0
2
+2
IronDog
IronDog
Питер Норвиг: Сегодня самые насущные вопросы в области искусственного интеллекта ориентированы на человека
+14
1.4K
1.4K
22
0
nemirovskiy_av
nemirovskiy_av
Как белка с ИИ сэкономила нам 100 миллионов на обогащении руды
+46
5.2K
5.2K
15
20
+20
phillennium
phillennium
Два в одном: фронтенд и мобильная разработка на EPAM Meet Up
+6
2.3K
2.3K
16
0
atomlib
atomlib
Who controls App Store: Martians or AI? Closed session of Russia's Federation Council and Apple leaked online
+17
2.4K
2.4K
4
0
stas_tibekin
stas_tibekin
Новый тренд: применение искусственного интеллекта и машинного обучения в контексте DevOps
0
1K
1K
13
1
+1
TechRecruiter
TechRecruiter
Cover Letter для Data Scientist: как написать сопроводительное письмо
+3
1.6K
1.6K
23
2
+2
EgoAILab
EgoAILab
TextIT: полезные инструменты для вашего сайта, интернет-магазина, мобильного приложения и чат-бота
0
909
909
19

In [104]:
habr_dict

{'Name': 'HuggingArtists | Генерируем текст песен с трансформером за 5 минут'}

In [137]:
for el in source_data:
    habr_dict = {}
    habr_dict['Name'] = el.find_element_by_class_name("tm-user-info__user").text
    habr_dict['Time'] = el.find_element_by_class_name("tm-article-snippet__datetime-published").text
    habr_dict['News'] = el.find_element_by_class_name("tm-article-snippet__title-link").text
    a = el.find_element_by_class_name("tm-article-snippet__title-link")
    habr_dict['Url'] = a.get_attribute('href')
    habr_list.append(habr_dict)
    #print(habr_dict)


In [139]:
my_df = pd.DataFrame(habr_list)
my_df

,Name,Time,News,Url
0,daniilshat,вчера в 13:02,Исследователи МТИ научили нейросеть предсказыв...,https://habr.com/ru/news/t/583210/
1,IronDog,12 октября в 16:16,Питер Норвиг: Сегодня самые насущные вопросы в...,https://habr.com/ru/post/583040/
2,nemirovskiy_av,12 октября в 16:11,Как белка с ИИ сэкономила нам 100 миллионов на...,https://habr.com/ru/company/nlmk/blog/580984/
3,phillennium,23 сентября в 12:15,Два в одном: фронтенд и мобильная разработка н...,https://habr.com/ru/company/epam_systems/blog/...
4,atomlib,17 сентября в 16:05,Who controls App Store: Martians or AI? Closed...,https://habr.com/ru/post/578574/
5,stas_tibekin,13 сентября в 13:10,Новый тренд: применение искусственного интелле...,https://habr.com/ru/company/nixys/blog/573456/
6,TechRecruiter,11 сентября в 14:50,Cover Letter для Data Scientist: как написать ...,https://habr.com/ru/post/577410/
7,EgoAILab,8 сентября в 15:26,"TextIT: полезные инструменты для вашего сайта,...",https://habr.com/ru/post/576888/
8,Spbwriter,8 сентября в 07:35,OmniLine и Chatme.ai будут создавать “умные” ч...,https://habr.com/ru/news/t/576802/
9,stas_tibekin,7 сентября в 10:31,Будущее DevOps: Основные тренды и прогнозы на ...,https://habr.com/ru/company/nixys/blog/573448/


In [141]:
client = MongoClient('localhost', 27017)
db = client['Habr_news_database']
collection = db.Habr_news_collection

In [142]:
db.collection.insert_many(my_df.to_dict('records'))

In [143]:
for doc in db.collection.find({}):
    print(doc)
    print('**************************')
    break

{'_id': ObjectId('616872fc5dc613c98598bfc0'), 'Name': 'daniilshat', 'Time': 'вчера в 13:02', 'News': 'Исследователи МТИ научили нейросеть предсказывать аварийные участки дорог', 'Url': 'https://habr.com/ru/news/t/583210/'}
**************************
